In [10]:
from ILP import MinimalRewiringILP
import numpy as np

In [18]:
init_wiring = np.array([[5,1,2],
                        [1,2,2],
                        [1,2,2]])

In [25]:
MR = MinimalRewiringILP(init_wiring)
p = MR.rewire("spine",5)
for i in p:
    print(i) 
print('\nCurrent wiring:')
print(MR.current_wiring)

('DISCONNECT', 0, 0)
('CONNECT', 0, 1)
('DISCONNECT', 0, 0)
('CONNECT', 0, 3)
('DISCONNECT', 0, 0)
('CONNECT', 0, 3)
('DISCONNECT', 1, 2)
('CONNECT', 1, 3)
('DISCONNECT', 2, 1)
('CONNECT', 2, 3)

Current wiring:
[[2 2 2 2]
 [1 2 1 1]
 [1 1 2 1]]


[[2 2 2 2]
 [1 2 1 1]
 [1 1 2 1]]


In [4]:
import random, math, json, sys
from ILP import MinimalRewiringILP
from network import Network 


class Controller(object):
  def __init__(self): 

    self.switches = {}      # Switches 
    self.num_switches = -1  # track that all switches have reported
    self.priority = 1100    # current max priority 
    self.numH = None        # number of hosts  
    self.verbose = 0        # reporting details 

    self.network = self.initial_network1()  
    self.wiring, self.agg_key, self.core_key = self.network.core_agg_wiring() 
    self.minwiring = MinimalRewiringILP(self.wiring)

  def add_switch(self, level, nports, pace=1):
    """ Add spine or server block switch with /nports/ complete with all 
    necessary rewiring and rerouting

    Args:
          level (string): "spine" or "server"
          nports (int): number of ports on switch
          pace (int): number of instructions to install before rerouting

    """
    self.network.max_sid += 1 
    sid = self.network.max_sid 

    if level == 'spine':
      stype = 'core'
      self.core_key[len(self.core_key)] = sid
    else: 
      stype = 'agg'
      self.agg_key[len(self.agg_key)] = sid
        
    self.network.add_switch(sid, nports, stype)

    instructions = self.minwiring.rewire(level, nports)
    print(self.minwiring.current_wiring)
    sys.stdout.flush()
    
    for i, instr in enumerate(instructions):
      # add or delete link from network state and mininet topology
      a_id = self.agg_key[instr[1]]
      c_id = self.core_key[instr[2]]
      if instr[0] == "CONNECT":
        self.network.add_link(a_id, c_id, 1) 
        print("Adding 1 link between switch {} and {}".format(a_id, c_id))
      elif instr[0] == "DISCONNECT":
        self.network.remove_link(a_id, c_id, 1) 
        print("Removing 1 link between switch {} and {}".format(a_id, c_id))

      if i % pace == 0:
        # reroute every pace instructions
#         routes = self.network.route_ecmp() 
        pass
            
    # Final rerouting before exit        
    routes = self.network.route_ecmp() 

  def initial_network1(self):
    """ Sample starting network onto which we'll add nodes """
    net = Network()

    for i in range(1,9):
      net.add_switch(i,4,'host')
    for i in range(9,13):
      net.add_switch(i,4,'edge')
    for i in range(13,17):
      net.add_switch(i,5,'agg')
    for i in range(17,19):
      net.add_switch(i,6,'core')

    # host:edge links
    h_e = [(1,9,1),(2,9,1),(3,10,1),(4,10,1),(5,11,1),(6,11,1),(7,12,1),(8,12,1)] 
    # edge:agg links
    e_a = [(13,9,1),(14,9,1),(13,10,1),(14,10,1),(15,11,1),(16,11,1),(15,12,1),(16,12,1)]
    # agg:core links
    a_c = [(13,17,2),(14,17,1),(13,18,1),(14,18,2),(15,17,2),(16,17,1),(15,18,1),(16,18,2)] 

    for link in h_e + e_a + a_c:
      net.add_link(*link)
    return net



In [5]:
c = Controller()

In [6]:
c.add_switch('spine', 5,2)

[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
Removing 1 link between switch 13 and 17
Adding 1 link between switch 13 and 19
Removing 1 link between switch 14 and 18
Adding 1 link between switch 14 and 19
Removing 1 link between switch 15 and 17
Adding 1 link between switch 15 and 19
Removing 1 link between switch 16 and 18
Adding 1 link between switch 16 and 19


In [7]:
c.network.routes

defaultdict(dict,
            {9: {1: 1, 2: 2, 3: 3, 4: 4, 5: 3, 6: 4, 7: 3, 8: 4},
             10: {1: 3, 2: 4, 3: 1, 4: 2, 5: 3, 6: 4, 7: 3, 8: 4},
             11: {1: 3, 2: 4, 3: 3, 4: 4, 5: 1, 6: 2, 7: 3, 8: 4},
             12: {1: 3, 2: 4, 3: 3, 4: 4, 5: 3, 6: 4, 7: 1, 8: 2},
             13: {1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 4, 7: 5, 8: 3},
             14: {1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 4, 7: 5, 8: 3},
             15: {1: 3, 2: 4, 3: 5, 4: 3, 5: 1, 6: 1, 7: 2, 8: 2},
             16: {1: 3, 2: 4, 3: 5, 4: 3, 5: 1, 6: 1, 7: 2, 8: 2},
             17: {1: 1, 2: 3, 3: 3, 4: 1, 5: 4, 6: 4, 7: 4, 8: 4},
             18: {1: 2, 2: 2, 3: 1, 4: 1, 5: 5, 6: 4, 7: 4, 8: 5},
             19: {1: 2, 2: 1, 3: 1, 4: 2, 5: 4, 6: 3, 7: 4, 8: 3}})

In [10]:
print(c.network.switches[1])

Switch 1, of type 'host', with 4 ports
Has 1 links, with 1 of them pointing up
Linked to Switch 9 at ports [1], Switch 1 at ports []


In [ ]:
import numpy as np

In [26]:
a =np.array([[2, 1],
 [1, 2],
 [2, 1],
 [1, 2]])


In [27]:
a.shape

(4, 2)

In [28]:
a

array([[2, 1],
       [1, 2],
       [2, 1],
       [1, 2]])

In [8]:
s = [4,5,6,7,8]

In [9]:
s[-3:]

[6, 7, 8]

In [10]:
s[:-3]

[4, 5]